In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 3.2 : Run CCDC over a tiled grid

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/3.2-CCDC_and_Scaling_Over_Large_Areas.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3.2-CCDC_and_Scaling_Over_Large_Areas.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3.2-CCDC_and_Scaling_Over_Large_Areas.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook uses Landsat in the Continuous Change Detection and Classification (CCDC) temporal segmentation algorithm over a tiled grid in order to minimize memory errors

GEE can run out of memory for processes that use a deep stack of data, such as CCDC. Breaking exports into tiles can help avoid memory errors.

Learn more about CCDC:
- [CCDC Publication](https://www.sciencedirect.com/science/article/abs/pii/S0034425714000248).
- [GEE CCDC Documentation](https://developers.google.com/earth-engine/apidocs/ee-algorithms-temporalsegmentation-ccdc)

### Objective

In this tutorial, you learn how to create and manipulate CCDC outputs.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:



**The steps performed include:**

- Create tile mesh grid over a study area
- Create CCDC outputs over the mesh grid
- Manipulate EE array image objects to get meaningful data out of CCDC outputs - change detection

**Learning objectives include:**
- Users will understand the purpose of temporal segmentation as implemented in CCDC.
- Users will understand key parameters in CCDC algorithm. 
- Users will be able to generate and manipulate array outputs from CCDC. 


# Before you begin

### Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Set a folder to use for all exports under `export_path_root` 
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the pre-baked folder (where outputs have already been created). 
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as it was. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [1]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com'
pre_baked_path_root = 'projects/rcr-gee/assets/lcms-training'
export_path_root  = pre_baked_path_root

print('Done')

Done


# Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

Note that for this module, we're also importing the `geeViz.changeDetectionLib as changeDetectionLib`. We will use this library later to 

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import inspect
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
Done


## Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [3]:
# Create folder and a collection and make public

export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'
aml.create_asset(export_ccdc_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_ccdc_collection,writers = [],all_users_can_read = True,readers = [])

# aml.batchCopy('projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4',export_ccdc_collection,outType = 'imageCollection')
# aml.batchDelete('projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4')
# print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-3_CCDC']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC


## View tiles used in current CONUS LCMS workflow

Currently, we run LCMS for the Continental US (CONUS), Coastal Alaska, Hawaii, and Puerto Rico / the US Virgin Islands. 

Run the codeblock below. 

In the window that appears, click the box next to the layer names to turn on the layers.

In [7]:
#Reset port if necessary
Map.port = 1234

Map.clearMap()
Map.proxy_url = workbench_url
# First, we'll view the tiles used in the current CONUS LCMS workflow
lcms_CONUS_composites = ee.ImageCollection('projects/lcms-tcc-shared/assets/Composites/Composite-Collection-yesL7-1984-2020')\
                                                .filter(ee.Filter.calendarRange(2022,2022,'year'))

# Pull the geometry of each tile in the composites
lcms_composites_tile_geo = lcms_CONUS_composites.map(lambda f:ee.Feature(f.geometry()).copyProperties(f,['studyAreaName']))

# Add the tiles and a composite for reference
Map.addLayer(lcms_CONUS_composites.mosaic(),getImagesLib.vizParamsTrue10k,'Example CONUS 2022 LCMS Composite')
Map.addLayer(lcms_composites_tile_geo,{},'LCMS Composite Tile Geometry')

Map.centerObject(lcms_composites_tile_geo)
Map.turnOnInspector()
Map.view()

Adding layer: Example CONUS 2022 LCMS Composite
Adding layer: LCMS Composite Tile Geometry
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:1234/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  1234
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:56] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:57] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:57] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:57] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:16:57] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 22:

### What do we want people to notice about the tiles? 
Note that these tiles are much larger than the area that we're working on with puerto ric0

## Create tiles of various sizes

Here is an example of how to create a pyramid of tiles at various scales
Below, you'll write a function to generate a grid of a set size and add it to the map. Then, you'll use that function to create tile sets of various sizes. 

To determine what size tile you would use, 
Generally you would start with the biggest tile possible and work your way down till you stop having memory issues
Currently, LCMS uses 480km tiles (with a 900m buffer) for most processing

In [8]:
Map.clearMap()

# set study area and projection
lcms_CONUS_studyArea = ee.FeatureCollection('projects/lcms-292214/assets/CONUS-Ancillary-Data/conus')
lcms_CONUS_projection = lcms_CONUS_composites.first().projection()

def getGrid(studyArea,projection,size):
  grid = studyArea.geometry().coveringGrid(projection.atScale(size))
  Map.addLayer(grid,{},'Tile Grid {}m'.format(size))
  return grid

grid480= getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,480000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,240000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,120000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,60000)
Map.addLayer(lcms_CONUS_studyArea,{},'LCMS CONUS Study Area')

Map.turnOnInspector()
Map.view()


Adding layer: Tile Grid 480000m
Adding layer: Tile Grid 240000m
Adding layer: Tile Grid 120000m
Adding layer: Tile Grid 60000m
Adding layer: LCMS CONUS Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [10/Aug/2023 22:17:33] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Takeaway from this map
We can scale in all different sizes, scale changes number of tiles exponentially (?)
Tiles might be too large or too small for area of interest depending

## Using Tiles
As an example of how to use this approach, we first need to get each available tile into a list
Then we would iterate across each tile, get the data, and export, etc...

Zoom to the northwestern US to see these tiles

In [27]:
Map.clearMap()

ids = grid480.limit(2).aggregate_histogram('system:index').keys().getInfo()
for id in ids:
  # Get the tile and clip it to the study area and then buffer
  tile = grid480.filter(ee.Filter.eq('system:index',id)).geometry().intersection(lcms_CONUS_studyArea,240,lcms_CONUS_projection).buffer(900)
  Map.addLayer(tile,{},'Tile {}'.format(id))

Map.centerObject(tile)
Map.view()



Adding layer: Tile -3,-6
Adding layer: Tile -4,-6
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [15/Aug/2023 20:40:05] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## Example CCDC run

### Memory considerations
Since CCDC is the most memory intensive algorith we use, you are the most likely to need to use this approach when running CCDC

If you run it from 1984 to 2023, even using just Landsat (rather than Landsat and Sentinel-2), you are much more likely to run out of memory

### Set up tiles for example

We'll use the full Puerto Rico and US Virgin Islands LCMS study area, and will run our analysis from 1984-2023.

In [11]:
# Set the size (in meters) of the tiles
tileSize = 60000


# clear the map
Map.clearMap()

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))


Map.addLayer(studyArea,{},'Study Area')
Map.turnOnInspector()
Map.centerObject(studyArea)
Map.view()

Adding layer: Tile Grid 60000m
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [10/Aug/2023 22:39:12] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Define parameters for Landsat imagery 

First let's set up some Landsat and CCDC parameters.

We'll use the full Puerto Rico and US Virgin Islands LCMS study area, and will run our analysis from 1984-2023.

All of the parameters below you should remember from module 2

In [12]:
# We'll use the full Puerto Rico and US Virgin Islands LCMS study area
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

#Specify start and end years for all analyses
#More than a 3 year span should be provided for time series methods to work
#well. If providing pre-computed stats for cloudScore and TDOM, this does not
#matter
startYear = 1984
endYear = 2023

#Update the startJulian and endJulian variables to indicate your seasonal
#constraints. This supports wrapping for tropics and southern hemisphere.
#If using wrapping and the majority of the days occur in the second year, the system:time_start will default
#to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
#startJulian: Starting Julian date
#endJulian: Ending Julian date
startJulian = 1
endJulian = 365

#Choose whether to include Landat 7
#Generally only included when data are limited
includeSLCOffL7 = True

### Set export parameters

#### Export bands
First, set Which bands/indices to export
These will not always be used to find breaks - that is specified below in the ccdcParams

Options are: ["blue","green","red","nir","swir1","swir2","NDVI","NBR","NDMI","NDSI","brightness","greenness","wetness","fourth","fifth","sixth","tcAngleBG"]

Be sure that any bands in `ccdcParams.breakpointBands` parameter, which we'll set next, are in this list

#### Export projection and scale


In [10]:
# set export bands
exportBands = ["blue","green","red","nir","swir1","swir2","NDVI"]

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

#### Visualization parameters

Adjusting settings for how the visualizer displays false color imagery

In [10]:
# set viz params
getImagesLib.vizParamsFalse['min']=0.15
getImagesLib.vizParamsFalse['max']=0.8

### Set CCDC parameters

One sentence summary of CCDC
Link out to CCDC documentation

**CCDC Parameters include:**
| Argument             | Type                    | Details                                                                                                                                                                                                                                |
|----------------------|-------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| collection           | ImageCollection         | Collection of images on which to run CCDC.                                                                                                                                                                                             |
| breakpointBands      | List, default: None     | The name or index of the bands to use for change detection. If unspecified, all bands are used.                                                                                                                                        |
| tmaskBands           | List, default: None     | The name or index of the bands to use for iterative TMask cloud detection. These are typically the green band and the SWIR1 band. If unspecified, TMask is not used. If specified, 'tmaskBands' must be included in 'breakpointBands'. |
| minObservations      | Integer, default: 6     | The number of observations required to flag a change.                                                                                                                                                                                  |
| chiSquareProbability | Float, default: 0.99    | The chi-square probability threshold for change detection in the range of [0, 1]                                                                                                                                                       |
| minNumOfYearsScaler  | Float, default: 1.33    | Factors of minimum number of years to apply new fitting.                                                                                                                                                                               |
| dateFormat           | Integer, default: 0     | The time representation to use during fitting: 0 = jDays, 1 = fractional years, 2 = unix time in milliseconds. The start, end and break times for each temporal segment will be encoded this way.                                      |
| lambda               | Float, default: 20      | Lambda for LASSO regression fitting. If set to 0, regular OLS is used instead of LASSO. 20 would be if input data was scaled 0-10000. If 0-1 reflectance, 20 would become 0.002                                                                                                                                                |
| maxIterations        | Integer, default: 25000 | Maximum number of runs for LASSO regression convergence. If set to 0, regular OLS is used instead of LASSO.                                                                                                                            |
* Run the code chunk below to set the parameters that you'll use in the CCDC model.


In [3]:
ccdcParams ={
  'breakpointBands':['green','red','nir','swir1','swir2','NDVI'],
  'tmaskBands' : None,
  'minObservations': 6,
  'chiSquareProbability': 0.99,
  'minNumOfYearsScaler': 1.33,
  'lambda': 0.002,# Since our reflectance data is 0-1 and not 0-10000, we divide 20 by 10000
  'maxIterations' : 25000,
  'dateFormat' : 1
};

#### Function to remove any extremely high band / index values

That might be artifacts and result in errors

In [17]:
# Write function to Remove any extremely high band/index values
def removeGT1(img):
  lte1 = img.select(['blue','green','nir','swir1','swir2']).lte(1).reduce(ee.Reducer.min());
  return img.updateMask(lte1);

### Iterate across each tile and run CCDC

    RUN EXAMPLE ACROSS JUST ONE TILE FIRST AND INSPECT -- run an example to show fitted curves for at least a small subset
    Example lives in the geeView package 

Now, we'll iterate across each tile and run CCDC
As an example of how to use this approach to create composites

In [17]:
##### Run CCDC

# list ids
#ids = grid.aggregate_histogram('system:index').keys().getInfo()
#ids = grid.aggregate_histogram('system:index').keys().first().getInfo()

#iterate over ids
for id in ids:
    print(id)
    # Get the tile and buffer it so there are no missing pixels at tile edges
    tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)
    
    # Map.addLayer(tile,{},'Tile {}'.format(id))
    
    processedScenes = getImagesLib.getProcessedLandsatScenes(studyArea = tile,startYear = startYear, endYear = endYear,
                                                        startJulian = startJulian,endJulian = endJulian,
                                                        includeSLCOffL7 = includeSLCOffL7).select(exportBands)
    processedScenes = processedScenes.map(removeGT1)
    # print(processedScenes.size().getInfo())
    
    #Set the scene collection in the ccdcParams
    ccdcParams['collection'] = processedScenes

    #Run CCDC
    ccdc = ee.Image(ee.Algorithms.TemporalSegmentation.Ccdc(**ccdcParams))
    ccdc = ccdc.set({'startYear':startYear,
                     'endYear':endYear,
                     'startJulian':startJulian,
                     'endJulian':endJulian,
                     'TileSize':tileSize,
                     'TileID':id})
    
    
    exportName = 'CCDC_Tile-{}m_ID{}_yrs{}-{}_jds{}-{}'.format(tileSize,id.replace(',','-'),startYear,endYear,startJulian,endJulian)
    exportPath = f'{export_ccdc_collection}/{exportName}'
    print(exportPath)

    getImagesLib.exportToAssetWrapper(ccdc,exportName,exportPath,{'.default':'sample'},tile,scale,crs,transform,overwrite=False)

90,54
Get Processed Landsat: 
Start date: Jan 01 1984 , End date: Dec 31 2023
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC/CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365
pyramiding object: {'.default': 'sample'}
CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
91,52
Get Processed Landsat: 
Start date: Jan 01 1984 , End date: Dec 31 2023
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors fo

### Task tracking

Can track tasks here or at https://code.earthengine.google.com/tasks
If you'd like to track the tasks, use the code below

In [67]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(exportPathRoot, type = 'imageCollection')

print('done')

done


### Inspect outputs - whole map 

Bring in the outputs and mosaic them into a single image. We will use this image later. 

NOTE: color scheme of output in viewer doesn't match legend 

In [16]:
Map.clearMap()

# Bring in the outputs and mosaic them into a single image
ccdcImg = ee.ImageCollection(export_ccdc_collection).mosaic()
Map.addLayer(ccdcImg,{},'CCDC Raw Image')

Map.turnOnInspector()
Map.view()

Adding layer: CCDC Raw Image
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:58] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 19:43:59] "

### Use CCDC to detect change

We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients

While CCDC is not as good at change detection as LandTrendr, the breaks can be used for detecting changes in phenology.

Below, we'll inspect the `ccdcChangeDetection` function in the `changeDetectionLib` to see what kind of parameters and inputs we'll need. 

In [14]:
# print option
print(inspect.getsource(changeDetectionLib.ccdcChangeDetection))

def ccdcChangeDetection(ccdcImg,bandName):
  magKeys = ['.*_magnitude']
  tBreakKeys = ['tBreak']
  changeProbKeys = ['changeProb']
  changeProbThresh = 1

  #Pull out pieces from CCDC output
  magnitudes = ccdcImg.select(magKeys)
  breaks = ccdcImg.select(tBreakKeys)
  
  #Map.addLayer(breaks.arrayLength(0),{'min':1,'max':10});
  changeProbs = ccdcImg.select(changeProbKeys)
  changeMask = changeProbs.gte(changeProbThresh)
  magnitudes = magnitudes.select(bandName + '.*')

  
  #Sort by magnitude and years
  breaksSortedByMag = breaks.arraySort(magnitudes)
  magnitudesSortedByMag = magnitudes.arraySort()
  changeMaskSortedByMag = changeMask.arraySort(magnitudes)
  
  breaksSortedByYear = breaks.arraySort()
  magnitudesSortedByYear = magnitudes.arraySort(breaks)
  changeMaskSortedByYear = changeMask.arraySort(breaks)
  
  #Get the loss and gain years and magnitudes for each sorting method
  highestMagLossYear = breaksSortedByMag.arraySlice(0,0,1).arrayFlatten([['loss_year']])
  highestM

#### Takeaways from the ccdcChangeDetection function
Some notes and highlights with things to pay attention to in the function

### Set change detection parameters for CCDC algorithm
This function allows us to manipulate CCDC outputs in array format in order to get meaningful information -- that is, information that's more directly useful than a modeled spectral trajectory. 

#### Specify which band to use for loss and gain.

This is most important for the loss and gain magnitude since the year of change will be the same for all years
`changeDetectionBandName = 'NDVI'`

Choose whether to show the most recent (`'mostRecent'`) or highest magnitude (`'highestMag'`) CCDC break
`sortingMethod = 'mostRecent'`

In [18]:
#Specify which band to use for loss and gain.
#This is most important for the loss and gain magnitude since the year of change will be the same for all years
changeDetectionBandName = 'NDVI'

# Choose whether to show the most recent ('mostRecent') or highest magnitude ('highestMag') CCDC break
sortingMethod = 'mostRecent'

### Run CCDC change detection and inspect outputs

We will not look at more useful ways of visualizing CCDC outputs
First, we will extract the change years and magnitude

Double click on map to see raw years of loss and gain breaks
Notice as you zoom in the layers change since GEE is processing outputs at a given pyramid level

In [19]:
#We will not look at more useful ways of visualizing CCDC outputs
#First, we will extract the change years and magnitude
changeObj = changeDetectionLib.ccdcChangeDetection(ccdcImg,changeDetectionBandName);

# clear map
Map.clearMap()

# add new layers to map
Map.addLayer(changeObj[sortingMethod]['loss']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.addLayer(changeObj[sortingMethod]['loss']['mag'],{'min':-0.5,'max':-0.1,'palette':changeDetectionLib.lossMagPalette},'Loss Mag',False);
Map.addLayer(changeObj[sortingMethod]['gain']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.gainYearPalette},'Gain Year');
Map.addLayer(changeObj[sortingMethod]['gain']['mag'],{'min':0.05,'max':0.2,'palette':changeDetectionLib.gainMagPalette},'Gain Mag',False);

Map.turnOnInspector()
Map.view()


Adding layer: Loss Year
Adding layer: Loss Mag
Adding layer: Gain Year
Adding layer: Gain Mag
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [15/Aug/2023 19:45:27] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -




We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
This function is useful to get the **harmonic model coefficient values** for a given date image 
`changeDetectionLib.getCCDCSegCoeffs`

In [26]:
# We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
# This function is useful to get the harmonic model coefficient values for a given date image 
print(inspect.getsource(changeDetectionLib.getCCDCSegCoeffs) )
Map.clearMap()

fillGaps = False
segCoeffs = changeDetectionLib.getCCDCSegCoeffs(ee.Image(2015.5), ccdcImg, fillGaps)
Map.addLayer(segCoeffs, {}, 'Seg Coeffs')
Map.addLayer(ccdcImg, {}, 'Raw Img')

Map.turnOnInspector()
Map.view()




def getCCDCSegCoeffs(timeImg,ccdcImg,fillGaps):
  coeffKeys = ['.*_coefs']
  tStartKeys = ['tStart']
  tEndKeys = ['tEnd']
  tBreakKeys = ['tBreak']
  
  #Get coeffs and find how many bands have coeffs
  coeffs = ccdcImg.select(coeffKeys)
  bns = coeffs.bandNames()
  nBns = bns.length()
  harmonicTag = ee.List(['INTP','SLP','COS1','SIN1','COS2','SIN2','COS3','SIN3'])

   
  #Get coeffs, start and end times
  coeffs = coeffs.toArray(2)
  tStarts = ccdcImg.select(tStartKeys)
  tEnds = ccdcImg.select(tEndKeys)
  tBreaks = ccdcImg.select(tBreakKeys)
  
  #If filling to the tBreak, use this
  tStarts = ee.Image(ee.Algorithms.If(fillGaps,tStarts.arraySlice(0,0,1).arrayCat(tBreaks.arraySlice(0,0,-1),0),tStarts))
  tEnds = ee.Image(ee.Algorithms.If(fillGaps,tBreaks.arraySlice(0,0,-1).arrayCat(tEnds.arraySlice(0,-1,None),0),tEnds))
  
  
  #Set up a mask for segments that the time band intersects
  tMask = tStarts.lt(timeImg).And(tEnds.gte(timeImg)).arrayRepeat(1,1).arrayRepeat(2,1)
  coeffs = 

127.0.0.1 - - [15/Aug/2023 20:35:25] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
This function is useful to get the **predicted values** for a given date
`changeDetectionLib.simpleCCDCPrediction`

In [28]:
# We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
# This function is useful to get the predicted values for a given date
print(inspect.getsource(changeDetectionLib.simpleCCDCPrediction) )

def simpleCCDCPrediction(img,timeBandName,whichHarmonics,whichBands):
  #Unit of each harmonic (1 cycle)
  omega = ee.Number(2.0).multiply(math.pi)

  #Pull out the time band in the yyyy.ff format
  tBand = img.select([timeBandName])
  
  #Pull out the intercepts and slopes
  intercepts = img.select(['.*_INTP'])
  slopes = img.select(['.*_SLP']).multiply(tBand)
  
  #Set up the omega for each harmonic for the given time band
  tOmega = ee.Image(whichHarmonics).multiply(omega).multiply(tBand)
  cosHarm = tOmega.cos()
  sinHarm = tOmega.sin()
  
  #Set up which harmonics to select

  harmSelect = ee.List(whichHarmonics).map(lambda n: ee.String('.*').cat(ee.Number(n).format()))
  
  #Select the harmonics specified
  sins = img.select(['.*_SIN.*'])
  sins = sins.select(harmSelect)
  coss = img.select(['.*_COS.*'])
  coss = coss.select(harmSelect)
  
  #Set up final output band names
  outBns = ee.List(whichBands).map(lambda bn: ee.String(bn).cat('_predicted'))
  
  #Iterate across each ban

In [23]:
print(inspect.getsource(changeDetectionLib.predictCCDC) )
print(inspect.getsource(changeDetectionLib.simpleCCDCPredictionWrapper) )

def predictCCDC(ccdcImg,timeImgs,fillGaps,whichHarmonics):
  timeBandName = ee.Image(timeImgs.first()).select([0]).bandNames().get(0)
  #Add the segment-appropriate coefficients to each time image
  timeImgs = timeImgs.map(lambda img: getCCDCSegCoeffs(img,ccdcImg,fillGaps))
  
  #Predict across each time image
  return simpleCCDCPredictionWrapper(timeImgs,timeBandName,whichHarmonics)

def simpleCCDCPredictionWrapper(c,timeBandName,whichHarmonics):
  whichBands = ee.Image(c.first()).select(['.*_INTP']).bandNames().map(lambda bn: ee.String(bn).split('_').get(0))
  whichBands = ee.Dictionary(whichBands.reduce(ee.Reducer.frequencyHistogram())).keys().getInfo()
  out = c.map(lambda img: simpleCCDCPrediction(img,timeBandName,whichHarmonics,whichBands))
  return out



### Run CCDC without tiles




In [21]:
# Same workflow follows as with an untiled CCDC output
Map.clearMap()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

#Apply the CCDC harmonic model across a time series
#First get a time series of time images with a time step of 0.1 of a year
yearImages = changeDetectionLib.getTimeImageCollection(startYear,endYear,startJulian,endJulian,0.1);

#Then predict the CCDC models
fitted = changeDetectionLib.predictCCDC(ccdcImg,yearImages,fillGaps,whichHarmonics)
Map.addLayer(fitted.select(['.*_predicted']),{'bands':'swir1_predicted,nir_predicted,red_predicted','min':0.05,'max':0.6},'Fitted CCDC',True);


# Synthetic composites visualizing
# Take common false color composite bands and visualize them for the next to the last year

# First get the bands of predicted bands and then split off the name
fittedBns = fitted.select(['.*_predicted']).first().bandNames()
bns = fittedBns.map(lambda bn: ee.String(bn).split('_').get(0))

# Filter down to the next to the last year and a summer date range
syntheticComposites = fitted.select(fittedBns,bns)\
    .filter(ee.Filter.calendarRange(endYear-1,endYear-1,'year'))\
    .filter(ee.Filter.calendarRange(60,80)).first()

# Visualize output as you would a composite
Map.addLayer(syntheticComposites,getImagesLib.vizParamsFalse,'Synthetic Composite')


#visualize time images

Map.addLayer(yearImages, {}, "year images")
Map.turnOnInspector()
Map.view()



Adding layer: Fitted CCDC
Adding layer: Synthetic Composite
Adding layer: year images
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [15/Aug/2023 20:12:15] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
